# Test to make interactive plots with plotly
The (plotly)[https://plotly.com/python/] library boasts that it makes the ploting of interactive graphs easy. Lets test this!

In this test i will attempt to:
 - Plot the distribution of road lenghts as violin plots, cut upon which province the road is in

# 0: Load in data and packages

In [20]:
# Load in relevant packages
import geopandas as gpd
import pandas as pd
import seaborn as sn
import numpy as np
from pathlib import Path
from shapely.geometry import Point, Polygon, MultiPoint
import plotly


# Load in the data and inspect in
# Load a shape file of world lakes as a GeoDataFrame:
path_roman_roads_split = Path('C:\\Users\\mlind\\github\\Minerva_hackathon\\data\\split segments\\road_merge_split_08112023.shp')
roman_roads: gpd.GeoDataFrame = gpd.read_file(path_roman_roads_split)

# We need to save the projection and the units used for the geometry for future reference
projection_roman_roads = roman_roads.crs
units_roman_roads = roman_roads.crs.axis_info[0].unit_name

# We load in the provinces data
path_to_provinces_geojson = Path("C:\\Users\\mlind\\github\\Minerva_hackathon\\data\\provinces_valid.geojson")
roman_provinces: gpd.GeoDataFrame = gpd.read_file(path_to_provinces_geojson)

projection_provinces = roman_provinces.crs
units_provinces = roman_provinces.crs.axis_info[0].unit_name

# We check out the projection and units of the geometries of the different objects
print(projection_provinces, projection_roman_roads, units_provinces, units_roman_roads)


EPSG:3035 EPSG:3395 metre metre


# 1: Prepare the data

In [21]:
print(projection_provinces, projection_roman_roads, "The projections are not the same so we need to change the provinces projection to match the on of the roads")

# cast the provinces. By doing this we re-project all of the points. I think this i desirable?
roman_provinces.to_crs(crs=projection_roman_roads, inplace=True)

# update variables
projection_provinces = roman_provinces.crs
units_provinces = roman_provinces.crs.axis_info[0].unit_name

print(projection_provinces, projection_roman_roads, "The projections are now the same, yay!")

roman_roads_length_gpd = gpd.GeoDataFrame(
    data={"length" : roman_roads["geometry"].length, "name" : roman_roads["Name"]},
    geometry=roman_roads["geometry"]
)

# Now, we need to split the roads dataframe based on the geometry of the provinces. We define a costum function for it
def find_province_for_road(geometry_road: Polygon):
    for _, province in roman_provinces.iterrows() :
        if geometry_road.within(province["geometry"]):
            return province["province"]
    return pd.NA

roman_roads_length_gpd["province"] = roman_roads_length_gpd["geometry"].map(find_province_for_road)


EPSG:3035 EPSG:3395 The projections are not the same so we need to change the provinces projection to match the on of the roads
EPSG:3395 EPSG:3395 The projections are now the same, yay!


In [28]:
roads_missing_provinces: gpd.GeoDataFrame = roman_roads_length_gpd.loc[roman_roads_length_gpd["province"].isna()]
print(f"There are {len(roads_missing_provinces)/len(roman_roads_length_gpd) * 100}% of roads which we have not been able to assign a province")
print("Not bad, not terribly good either")

There are 8.198068625436614% of roads which we have not been able to assign a province
Not bad, not terribly good either


# 2: Plot the data